In [1]:
import numpy as np
import pandas as pd 

%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

In [2]:
TRAIN_PATH = 'datasets/train.csv'
TEST_PATH = 'datasets/test.csv'

train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

print('train shape:',train.shape)
print('test shape:',test.shape)

train shape: (1460, 81)
test shape: (1459, 80)


In [3]:
train_id = train['Id']
test_id = test['Id']
train.drop(columns='Id', inplace=True)
test.drop(columns='Id', inplace=True)

## Preprocessing: 
* missing values
* outliers
* transformation
* feature engineering
* feature selection

In [4]:
# isoler colonnes avec valeurs manquantes
missings = train.isna().sum()
missings = missings.loc[missings>0].sort_values(ascending=False)

print('Pourcentage de valeurs manquantes par variable')
100 * np.round(missings/len(train),3)

Pourcentage de valeurs manquantes par variable


PoolQC          99.5
MiscFeature     96.3
Alley           93.8
Fence           80.8
FireplaceQu     47.3
LotFrontage     17.7
GarageYrBlt      5.5
GarageType       5.5
GarageFinish     5.5
GarageQual       5.5
GarageCond       5.5
BsmtFinType2     2.6
BsmtExposure     2.6
BsmtFinType1     2.5
BsmtCond         2.5
BsmtQual         2.5
MasVnrArea       0.5
MasVnrType       0.5
Electrical       0.1
dtype: float64

PoolQC, MiscFeature, Alley, Fense et FirePlaceQu ont clairement beaucoup trop de valeurs manquantes. (>50%) 
* Pour <b>LotFrontage</b>, à vérifier si les valeurs NaN correspondent à une absence de valeur ou à 0, puisque aucun 0 n'est présent.
* <b>FireplaceQu</b> a des Na qui correspondent d'après le métier à une absence de Fireplace, donc c'est une information à prendre en compte. Cependant, celle-ci est déjà contenue dans FirePlaces. => Supprumer FireplaceQu
* <b>Fence</b> a des Na qui correspondent d'après le métier à une absence de Fence, donc c'est une information à prendre en compte.
* De même pour <b>Alley</b>, Na signifie 'No Alley access', ce qui est une information.
* De même pour <b>PoolQC</b>, Na signifie 'No Pool', ce qui est une information. Cependant, l'information Qu'il n'y a pas de piscine est déjà contenue dans la variable PoolArea qui indique 0 quand la sruface est nulle (donc inexistante)

1) Remplacer les 'Na' par des No 'X' pour les X variables (Fence et Alley).

2) <b>GarageX :</b> ['GarageYrBlt','GarageType','GarageFinish','GarageQual','GarageCond'] ont quelques valeurs manquantes (5%) et visiblement les mêmes. La plupart du temps le Na signifie 'No Garage' et cette information est contenue déjà dans GarageCars ou GarageArea. On peut les mettre de coté pour l'instant. Même conclusion pour les <b>BsmtX</b>.

3) Donc on supprime les colonnes : LotFrontage, GarageX,BsmtX, PoolQC, MasVnrArea, MasVnrType  et FireplaceQu

4) fill Electrical

In [5]:
# replace NaN de Alley et Fence 
cols_change = ['Alley','Fence']

for col in cols_change:
    train[col].fillna(value='None',inplace=True)

In [6]:
supp_cols = ['GarageYrBlt','GarageType','GarageFinish','GarageQual','GarageCond','BsmtFinType2','BsmtExposure',
             'BsmtFinType1','BsmtCond','BsmtQual' ,
             'LotFrontage','PoolQC','MasVnrArea','MasVnrType','FireplaceQu','MiscFeature' ]

train.drop(columns=supp_cols,inplace=True)
train.shape

(1460, 64)

In [7]:
# Fill Electrical

train['Electrical'].fillna(value=train['Electrical'].mode()[0],inplace=True)

Transform des valeurs numeriques en categories

In [8]:
#MSSubClass=The building class
train['MSSubClass'] = train['MSSubClass'].apply(str)

#Year and month sold are transformed into categorical features.
train['YrSold'] = train['YrSold'].astype(str)
train['MoSold'] = train['MoSold'].astype(str)

Maintenant, pre-preocessing des valeurs numériques.

In [9]:
numerical_data = train._get_numeric_data()
numerical_data.columns

Index(['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'SalePrice'],
      dtype='object')

In [10]:
numerical_data.skew().sort_values(ascending=False)

MiscVal          24.476794
PoolArea         14.828374
LotArea          12.207688
3SsnPorch        10.304342
LowQualFinSF      9.011341
KitchenAbvGr      4.488397
BsmtFinSF2        4.255261
ScreenPorch       4.122214
BsmtHalfBath      4.103403
EnclosedPorch     3.089872
OpenPorchSF       2.364342
SalePrice         1.882876
BsmtFinSF1        1.685503
WoodDeckSF        1.541376
TotalBsmtSF       1.524255
1stFlrSF          1.376757
GrLivArea         1.366560
BsmtUnfSF         0.920268
2ndFlrSF          0.813030
OverallCond       0.693067
TotRmsAbvGrd      0.676341
HalfBath          0.675897
Fireplaces        0.649565
BsmtFullBath      0.596067
OverallQual       0.216944
BedroomAbvGr      0.211790
GarageArea        0.179981
FullBath          0.036562
GarageCars       -0.342549
YearRemodAdd     -0.503562
YearBuilt        -0.613461
dtype: float64

In [11]:
# toutes les varaibles au dessus de 1 => transformation
train[numerical_data.columns] = train[numerical_data.columns].apply(np.log1p)

Transformer categorie où l'ordre est important en variable quantitative discrete : 

In [12]:
cols_quali = ['ExterQual', 'ExterCond','HeatingQC', 'KitchenQual',
              'Functional', 'Fence', 'LandSlope','LotShape', 'PavedDrive', 'Street', 'Alley',
              'CentralAir', 'OverallCond', 'YrSold', 'MoSold']

quali_ordered = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0}

In [13]:
for col in cols_quali:
    print('\n',train[col].value_counts())


 TA    906
Gd    488
Ex     52
Fa     14
Name: ExterQual, dtype: int64

 TA    1282
Gd     146
Fa      28
Ex       3
Po       1
Name: ExterCond, dtype: int64

 Ex    741
TA    428
Gd    241
Fa     49
Po      1
Name: HeatingQC, dtype: int64

 TA    735
Gd    586
Ex    100
Fa     39
Name: KitchenQual, dtype: int64

 Typ     1360
Min2      34
Min1      31
Mod       15
Maj1      14
Maj2       5
Sev        1
Name: Functional, dtype: int64

 None     1179
MnPrv     157
GdPrv      59
GdWo       54
MnWw       11
Name: Fence, dtype: int64

 Gtl    1382
Mod      65
Sev      13
Name: LandSlope, dtype: int64

 Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64

 Y    1340
N      90
P      30
Name: PavedDrive, dtype: int64

 Pave    1454
Grvl       6
Name: Street, dtype: int64

 None    1369
Grvl      50
Pave      41
Name: Alley, dtype: int64

 Y    1365
N      95
Name: CentralAir, dtype: int64

 1.791759    821
1.945910    252
2.079442    205
2.197225     72
1.609438     57


In [14]:
for col in ['ExterQual', 'ExterCond','HeatingQC', 'KitchenQual']:
    train[col] = train[col].map(quali_ordered)

# Functionnal col : 
col = 'Functional'
train[col] = train[col].map({'Typ':7,'Min1':6,'Min2':5,'Mod':4,'Maj1':3,'Maj2':2,'Sev':1,'Sal':0})

# Fence col : 
col = 'Fence'
train[col] = train[col].map({'GdPrv':4,'MnPrv':3,'GdWo':2,'MnWw':1,'None':0})

# LandSlope col : 
col = 'LandSlope'
train[col] = train[col].map({'Gtl':2,'Mod':1,'Sev':0})

# LotShape col : 
col = 'LotShape'
train[col] = train[col].map({'Reg':3,'IR1':2,'IR2':1,'IR3':0})

# PavedDrive col : 
col = 'PavedDrive'
train[col] = train[col].map({'Y':2,'P':1,'N':0})

# Street
col = 'Street'
train[col] = train[col].map({'Pave':1,'Grvl':0})

# Alley
col = 'Alley'
train[col] = train[col].map({'Grvl':2,'Pave':1,'None':0})

Les autres variables peuvent être traitées comme des dummies.

In [15]:
train = pd.get_dummies(train)
train.shape

(1460, 224)

Validation : 

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train.drop(columns='SalePrice'),train['SalePrice'], test_size=0.2)

In [35]:
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR

sgd_reg = SGDRegressor()
sgd_reg.fit(X_train,y_train)
print('train score:', sgd_reg.score(X_train, y_train))
print('train score:', sgd_reg.score(X_val, y_val))

train score: -3.06227264276318e+17
train score: -7.219541904683063e+17


In [36]:
svm_reg = SVR()
svm_reg.fit(X_train,y_train)
print('train score:', svm_reg.score(X_train, y_train))
print('train score:', svm_reg.score(X_val, y_val))

C:\Users\clmen\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


train score: 0.9294665272161738
train score: 0.8165710611861889
